In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt

# I want to try to do everything in Pandas/Numpy this time, so let's make
# sure that we are able to see all columns and rows in our outputs.  

pd.set_option('display.max_columns',None)

In [2]:
# Let's first bring in the data into dataframes
# This will make my life a bit easier in the long run.

Teams = pd.read_excel('PD - NBA Results.xlsx',0)
October = pd.read_excel('PD - NBA Results.xlsx',1)
November = pd.read_excel('PD - NBA Results.xlsx',2)
December = pd.read_excel('PD - NBA Results.xlsx',3)
January = pd.read_excel('PD - NBA Results.xlsx',4)

# Quick housekeeping.  Due to COVID, many games in January were not played, so in order to protect
# our data's integrity, let's clean that up.  

January = January.dropna(subset=['Attend.'])
#    Goals
# -- Rank for each team based on number of wins
# -- Wins for each team
# -- Losses for each team
# -- Pct:  Win Percentage for each team
# -- Conf -- Number of conference wins (Only have 2, should be easy)
# -- Home -- Home game wins
# -- Away -- Away game wins
# -- L10 -- Wins '-' Losses
# -- Strk -- The current winning or losing streak

In [3]:
# Rank ---
# Based on what I see, the monthly columns all should identical and we
# will have to calculate across all months at each and every time.  

Games = [October, November, December, January]
Games = pd.concat(Games)

Games_All = [October, November, December, January]
Games_All = pd.concat(Games_All)

# Wins
# First step, let's find out who is winning, by using the where command for
# Numpy magic again!  Boom! 

Win = np.where(Games['PTS']>Games['PTS.1'], Games['Visitor/Neutral'], Games['Home/Neutral'])

# Let's update our master table, by adding the winner column to it!  
Games['Wins'] = Win

# Let's get the total count of wins.
Winning = Games['Wins'].value_counts()

In [4]:
# Let's get Home/Away Game Wins!  

Homegames = Games[Games['PTS']<Games['PTS.1']]
Homegames = Homegames['Wins'].value_counts()

# To pair the total home/away games played, with the wins, let's create 2 dataframes and join them together.  
Homegames = pd.merge(Games['Home/Neutral'].value_counts().to_frame(),Homegames,left_index=True, right_index=True)

# Let's reset the index, since the index is actually the Team name, we want to use (I could keep it and join on that, 
# but I special and will to that route, which has added a few extra lines of code.)  And let's sort our data.  
Homegames = Homegames.reset_index()
Homegames = Homegames.rename(columns={"index": "Team","Wins":"Home"})

# Let's concatenate the columns to create the home win/loss value and then drop the Total column.
Homegames['Home'] = Homegames['Home'].astype(str)+'-'+(Homegames['Home/Neutral']-Homegames['Home']).astype(str)
Homegames = Homegames.drop(columns='Home/Neutral')

# Awaygames is identical, except that the primary variable is Awaygames!
Awaygames = Games[Games['PTS']>Games['PTS.1']]
Awaygames = Awaygames['Wins'].value_counts()
Awaygames = pd.merge(Games['Visitor/Neutral'].value_counts().to_frame(),Awaygames,left_index=True, right_index=True)
Awaygames = Awaygames.reset_index()
Awaygames = Awaygames.rename(columns={"index": "Team","Wins":"Away"})
Awaygames['Away'] = Awaygames['Away'].astype(str)+'-'+(Awaygames['Visitor/Neutral']-Awaygames['Away']).astype(str)
Awaygames = Awaygames.drop(columns='Visitor/Neutral')

In [5]:
# If we look at what we have above,the team name is actually the index!  That cannot be.  Let's fix that.  
Winning = Winning.to_frame().reset_index()

# I don't like the column name of "Index", so lets fix that.
Winning = Winning.rename(columns={"index": "Team"})

In [6]:
# Losses for each team, let's get that too... Wait... can't I just recreate my scripts to make this easier?  Booyah!

Loser = np.where(Games['PTS']<Games['PTS.1'], Games['Visitor/Neutral'], Games['Home/Neutral'])
Games['Losses'] = Loser
Losses = Games['Losses'].value_counts()

In [7]:
# If we look at what we have above,the team name is actually the index!  That cannot be.  Let's fix that.  
Losses = Losses.to_frame().reset_index()

# I don't like the column name of "Index", so lets fix that.
Losses = Losses.rename(columns={"index": "Team"})

# Now we have the total losses of each team!  Done!

In [8]:
# Win percentages for each and every team!  
# First off, we need to figure out how many games each team played during the season.  Let's use that value_counts trick
# but with a slight caveat.  

Played = [Games['Visitor/Neutral'], Games['Home/Neutral']]
Played = pd.concat(Played)
Played = Played.value_counts()

In [9]:
# If we look at what we have above,the team name is actually the index!  That cannot be.  Let's fix that.  
Played = Played.to_frame().reset_index()

# I don't like the column name of "Index", so lets fix that.
Played = Played.rename(columns={"index": "Team", 0: "Games_Played"})

# With the number of games played, we can now calculate win percentage.  Consider this done!  

In [10]:
# Conference Wins!  For this one, we will need to do a quick join on our data, to populate the conference
# for each team accordingly.  

# Let's setup the conference information
Visitor = Teams.rename(columns={"Conference":"V_Conf"})
Visitor = Visitor.drop(columns=['Divison'])
Conference = pd.merge(Visitor, Games, left_on='Team', right_on = 'Visitor/Neutral')

In [11]:
# Same as the previous step, but for home teams
Home = Teams.rename(columns={"Conference":"H_Conf"})
Home = Home.drop(columns=['Divison'])
Conference = pd.merge(Home, Conference, left_on='Team', right_on = 'Home/Neutral')

In [12]:
# Let's clean the dataframe a bit and remove unnecessary columns
Conference = Conference.drop(columns=['Team_x','Team_y'])

In [13]:
# This step validates if it is truly a conference win, if both teams are in the same conference.
Conference = Conference[Conference['H_Conf'] == Conference['V_Conf']]

In [14]:
# Conference W/L per team
Conf_Wins = Conference['Wins'].value_counts()
Conf_Losses = Conference['Losses'].value_counts()

# Let's fix the index and create a dataframe
Conf_Wins = Conf_Wins.to_frame().reset_index()
Conf_Losses = Conf_Losses.to_frame().reset_index()

# I don't like the column name of "Index", so lets fix that.
Conf_Wins = Conf_Wins.rename(columns={"index": "Team", "Wins": "Conf_Wins"})
Conf_Losses = Conf_Losses.rename(columns={"index": "Team", "Losses": "Conf_Losses"})

# Last part, let's create the record for confernece wins and do one last clean up.
Conference = pd.merge(Conf_Wins, Conf_Losses, on='Team'); 
Conference['Conference_Record'] = Conference['Conf_Wins'].astype(str) +'-'+ Conference['Conf_Losses'].astype(str)
Conference = Conference.drop(columns=['Conf_Wins','Conf_Losses'])

In [15]:
# Last 10 Games time.  This one takes a bit of intuition, so let's take it slow. 
# We need our Dates to be squared away, so let's change the format

Games['Date'] = pd.to_datetime(Games['Date'])

# Let's now sort these values accordingly and make life better.
Games = Games.sort_values(by='Date', ascending=False)

# Finally, let's create a flag for wins, and losses.  We will use this later.  
Games['W_Flag'] = 1
Games['L_Flag'] = 0

In [16]:
# When attempting to select multiple columns from a dataframe, always use 2 brackets when you opt to use the 
# bracket method.  There are others, but just a heads up. 
# We are going to do a lot of steps right now to expedite things.

L10 = [Games[['Wins','Date','W_Flag']].rename(columns={"Wins":"Team","W_Flag":"Flag"}), Games[['Losses','Date','L_Flag']].rename(columns={"Losses": "Team","L_Flag":"Flag"})]

# Let's concatenate the list that we created above.  And notice that I made sure the column names were the same
# That is because it will make my life easier when it comes to aggregations.
L10 = pd.concat(L10, sort=True)

In [17]:
# We have all of our days that each team has played, so let's order them.  Almost there!
L10 = L10.sort_values(by='Date', ascending=False)

In [18]:
# With the most recent games at the top, and ordered accordingly, this will be sweet, just watch.
# This step groups our values by the team name in the Wins column and then increments based on the number
# of times that it appears.  This is a POWERFUL function.
Appearance = L10.groupby(['Team']).cumcount()+1

In [19]:
# Let's add the game counter to this
L10['Game_Count'] = Appearance

In [20]:
# Let's filter out any games beyond the 10th appearance (Remember, we are doing this backwards, so we are getting the 
# last 10 games that each team has played.)
L10 = L10[L10['Game_Count']<11]

In [21]:
# This step will create the total number of wins in the last 10 games for each team.
Last = L10.groupby(by=["Team"], as_index=False).sum()

In [22]:
# Let's create the appropriate column and get the formatting setup for the Last 10 metric.
Last['L10'] = Last['Flag'].astype(str) + '-' + (10-Last['Flag']).astype(str)

In [23]:
# Quick clean up on the data by dropping columns.  
Last = Last.drop(columns=['Flag','Game_Count']); Last

,Team,L10
0,Atlanta Hawks,2-8
1,Boston Celtics,8-2
2,Brooklyn Nets,3-7
3,Charlotte Hornets,3-7
4,Chicago Bulls,4-6
5,Cleveland Cavaliers,4-6
6,Dallas Mavericks,5-5
7,Denver Nuggets,7-3
8,Detroit Pistons,2-8
9,Golden State Warriors,4-6


In [24]:
# Let's do streak as well.  This one will use a similar technique as the Last 10, so let's use that pre-formatted data
Streak = L10

In [25]:
# Some Pandas sorting magic to get the streak.  
Streak = Streak.sort_values(['Team','Date'], ascending= (True, True))

In [26]:
# This step will count the number of times that a value appears consecutively, for a team, ergo our streak.

Streak['Streak'] = Streak['Flag'].groupby((Streak['Flag'] != Streak.groupby(['Team'])['Flag'].shift()).cumsum()).cumcount() + 1

In [27]:
# Since we sorted our records accordingly, let's get the last record for each team and drop the rest.  
Streak = Streak.drop_duplicates('Team', keep='last')

# Let's add the W/L to the flag column, which puts us at about 95% done.  Almost there!
Streak['Flag'] = Streak['Flag'].replace({0: 'L', 1: 'W'})

# And magically, we will concatenate the values and set the streak column appropriately.  
Streak['Streak'] = Streak['Flag'] + Streak['Streak'].astype(str)

In [28]:
# Before bringing all of this together, let's drop unnecesary columns.  
Streak = Streak.drop(columns=['Flag','Game_Count','Date'])

In [29]:
# We now have a bunch of aggregations, that will solve most if not all of the questions out there.  
# Let's bring all of these together to tell a single story

Metrics = pd.merge(Teams,Played,on='Team')
Metrics = pd.merge(Metrics, Conference, on='Team')
Metrics = pd.merge(Metrics,Winning, on='Team')
Metrics = pd.merge(Metrics, Losses, on='Team')
Metrics = pd.merge(Metrics, Awaygames, on='Team')
Metrics = pd.merge(Metrics, Homegames, on='Team')
Metrics = pd.merge(Metrics, Streak, on='Team')
Metrics = pd.merge(Metrics, Last, on='Team')

In [30]:
Metrics

,Team,Divison,Conference,Games_Played,Conference_Record,Wins,Losses,Away,Home,Streak,L10
0,Atlanta Hawks,Southeast,Eastern,36,5-19,8,28,4-16,4-12,W1,2-8
1,Boston Celtics,Atlantic,Eastern,33,18-5,25,8,11-6,14-2,W3,8-2
2,Brooklyn Nets,Atlantic,Eastern,34,10-9,16,18,7-11,9-7,L5,3-7
3,Charlotte Hornets,Southeast,Eastern,38,10-14,15,23,8-12,7-11,W2,3-7
4,Chicago Bulls,Central,Eastern,36,9-15,13,23,6-10,7-13,L3,4-6
5,Cleveland Cavaliers,Central,Eastern,35,6-21,10,25,4-13,6-12,L3,4-6
6,Detroit Pistons,Central,Eastern,36,9-18,13,23,5-13,8-10,W1,2-8
7,Indiana Pacers,Central,Eastern,36,15-10,22,14,7-10,15-4,L2,5-5
8,Miami Heat,Southeast,Eastern,35,18-4,25,10,9-9,16-1,L1,7-3
9,Milwaukee Bucks,Central,Eastern,37,20-3,32,5,14-3,18-2,W5,8-2


In [31]:
# Win Percentage should be easy now
Metrics['Win_Percentage'] = Metrics['Wins']/Metrics['Games_Played']
Metrics = Metrics.sort_values(by='Win_Percentage', ascending=False).round({'Win_Percentage': 3})

# Quick cleaning, by dropping unnecessary columns.  I probably could have been smarter and done this earlier, oh well.
# And a quick sort of columns, so that it looks like the output(ish).  
Metrics = Metrics.drop(columns=['Divison','Games_Played'])

In [32]:
# Let's re-arrange our columns to put everything in its place.  
Metrics['Rank'] = ""
Metrics = Metrics[["Rank","Team","Conference","Wins","Losses","Win_Percentage","Conference_Record","Home","Away","L10","Streak"]]

In [33]:
# Let's create the two outputs, which should finish this problem!  
Eastern = Metrics[Metrics['Conference'] == 'Eastern']
Western = Metrics[Metrics['Conference'] == 'Western']

In [37]:
# Let's put the rank on each of these areas to wrap this up.
Western.loc[:, ('Rank')] = (Western['Win_Percentage'].diff() != 0).cumsum()
Eastern.loc[:, ('Rank')] = (Eastern['Win_Percentage'].diff() != 0).cumsum()

# You will notice that this looks a bit different than how we have been storing values.  Rerun this row with these
# below, to see the warning message and some documentation on how to mitigate it.  
#Western['Rank']= (Western['Win_Percentage'].diff() != 0).cumsum()
#Eastern['Rank']= (Eastern['Win_Percentage'].diff() != 0).cumsum()

In [35]:
# Western, is DONE!
Western

,Rank,Team,Conference,Wins,Losses,Win_Percentage,Conference_Record,Home,Away,L10,Streak
20,1,Los Angeles Lakers,Western,28,7,0.800,20-4,13-4,15-3,6-4,W4
18,2,Houston Rockets,Western,24,11,0.686,14-7,13-4,11-7,7-3,W2
16,2,Denver Nuggets,Western,24,11,0.686,14-6,15-4,9-7,7-3,L1
19,3,Los Angeles Clippers,Western,25,12,0.676,16-9,15-4,10-8,5-5,L1
29,4,Utah Jazz,Western,23,12,0.657,14-7,13-3,10-9,9-1,W5
15,5,Dallas Mavericks,Western,22,13,0.629,13-6,10-8,12-5,5-5,L1
24,6,Oklahoma City Thunder,Western,20,15,0.571,14-11,12-6,8-9,9-1,W5
26,7,Portland Trail Blazers,Western,15,21,0.417,9-15,8-9,7-12,5-5,W1
28,8,San Antonio Spurs,Western,14,20,0.412,9-12,10-9,4-11,5-5,L2
25,9,Phoenix Suns,Western,14,21,0.400,9-17,7-12,7-9,3-7,W1


In [36]:
# Eastern is DONE!  Let me know if you have questions.  
Eastern

,Rank,Team,Conference,Wins,Losses,Win_Percentage,Conference_Record,Home,Away,L10,Streak
9,1,Milwaukee Bucks,Eastern,32,5,0.865,20-3,18-2,14-3,8-2,W5
1,2,Boston Celtics,Eastern,25,8,0.758,18-5,14-2,11-6,8-2,W3
8,3,Miami Heat,Eastern,25,10,0.714,18-4,16-1,9-9,7-3,L1
13,4,Toronto Raptors,Eastern,24,12,0.667,17-7,14-5,10-7,6-4,W1
12,5,Philadelphia 76ers,Eastern,23,14,0.622,16-8,16-2,7-12,3-7,L4
7,6,Indiana Pacers,Eastern,22,14,0.611,15-10,15-4,7-10,5-5,L2
2,7,Brooklyn Nets,Eastern,16,18,0.471,10-9,9-7,7-11,3-7,L5
11,8,Orlando Magic,Eastern,16,20,0.444,11-11,11-8,5-12,4-6,L1
3,9,Charlotte Hornets,Eastern,15,23,0.395,10-14,7-11,8-12,3-7,W2
6,10,Detroit Pistons,Eastern,13,23,0.361,9-18,8-10,5-13,2-8,W1
